In [ ]:
#import the libraries
import pandas as pd
import tqdm as tqdm
import re
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
# from textblob import TextBlob
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
import matplotlib.pyplot as plt
# import wordcloud
# from wordcloud import WordCloud
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
import itertools
import random
import warnings
warnings.filterwarnings('ignore')
import pickle
from numpy.random import choice
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#importing data and initialising stopwords
data = pd.read_csv("A1_dataset.csv")
stop_words = set(stopwords.words('english'))

In [ ]:
#initializing punctuations
punct = string.punctuation

In [ ]:
#preprocessing
stop_words = set(stopwords.words('english'))
def remove_white_spaces(sent):
    x = re.sub(r'\s+',' ',sent)#will remove extra spaces
    return(x)
def remove_unwanted_characters(sent):
    pat = re.compile('@[^\s]+')#atleast one character should be present after @ and valid username contains alphabets,digits and _
    cl_pat = re.sub(pat,'',sent)
    pat = re.compile("www.\S+")
    cl_pat = re.sub(pat, ' ', cl_pat)
    pat = re.compile("<.*?>")#html
    cl_pat = re.sub(pat, ' ', cl_pat)
    pat = re.compile("https?://\S+")#url
    cl_pat = re.sub(pat, ' ', cl_pat)
    cl_pat = ''.join([word for word in cl_pat if not word.isdigit()])
    cl_pat = re.sub(r"(\!)\1+", ' ! ', cl_pat)
    cl_pat = re.sub(r"(\?)\1+", ' ? ', cl_pat)
    cl_pat = re.sub(r"(\.)\1+", ' \. ', cl_pat)
    cl_pat = re.sub(r"(\\)\1+","",cl_pat)
    cl_pat = re.sub(r"\\","",cl_pat)
    cl_pat = word_tokenize(cl_pat)
    words = []
    for word in cl_pat:
        word = word.lower()
        if(word in stop_words or word in punct):
            continue
        words.append(word)
    
    cl_pat = " ".join(words)
    cl_pat = re.sub(r"we 're", "we are", cl_pat)
    cl_pat = re.sub(r"i \'m", "i am", cl_pat)
    cl_pat = re.sub(r"it \'s", "it is", cl_pat)
    cl_pat = re.sub(r"ca n\'t", "can't", cl_pat)
    cl_pat = re.sub(r"do n\'t", "don't", cl_pat)
    cl_pat = re.sub(r"\'m not", "i am not", cl_pat)
    cl_pat = re.sub(r"we 're", "we are", cl_pat)
    cl_pat = re.sub(r"wo n\'t", "will not", cl_pat)
    cl_pat = re.sub(r"\'s a", "is a", cl_pat)
    cl_pat = re.sub(r"does n\'t", "doesn't", cl_pat)
    cl_pat = re.sub(r"do n\'t", "don't", cl_pat)
    cl_pat = re.sub(r"\'ll be", "i will be", cl_pat)
    cl_pat = re.sub(r"gon na", "gonna", cl_pat)
    cl_pat = re.sub(r"got ta", "gotta", cl_pat)
    cl_pat = re.sub(r"\wan na ", "wanna", cl_pat)
#     cl_pat = re.sub(r"\'ll be", "i will be", cl_pat)
    

    return cl_pat

In [ ]:
#preprocessing the training dataa
for i in tqdm.tqdm(data.index):
    data.loc[i,'TEXT'] = remove_unwanted_characters(data.loc[i,'TEXT'])
    data.loc[i,'TEXT'] = remove_white_spaces(data.loc[i,'TEXT'])

100%|█████████████████████████████████████| 4287/4287 [00:01<00:00, 2145.06it/s]


In [ ]:
#adding starters and enders
for i in tqdm.tqdm(data.index):
    data.loc[i,'TEXT'] ='<s> ' + data.loc[i,'TEXT'] +' </s>'

100%|████████████████████████████████████| 4287/4287 [00:00<00:00, 10657.20it/s]


In [ ]:
#Finding the unigram count of the words in training corpus
unigrams = []
unigram_count = dict()

for i in tqdm.tqdm(data.index):
    sent = data.loc[i,'TEXT']
    tok = sent.split(' ')
    for i in range(len(tok)):
        
        try:
            unigram_count[tok[i]]+=1
        except:
            unigram_count[tok[i]] = 1


unigrams = list(unigram_count.keys())
for i in unigrams:
    if i in punct:
        unigrams.remove(i)
    if(i=="''"):
        unigrams.remove(i)
vocab = set(unigrams)
V = len(unigrams)

100%|████████████████████████████████████| 4287/4287 [00:00<00:00, 71053.38it/s]


In [ ]:
#Finding the bigram count of the words in training corpus
bigrams = []
bigram_count = dict()

for i in tqdm.tqdm(data.index):
    sent = data.loc[i,'TEXT']
    tok = sent.split(' ')
    for i in range(len(tok)-1):
        if(tok[i] =="''" or tok[i+1]=="''"):
            continue
        try:
            bigram_count[(tok[i],tok[i+1])] +=1
        except:
            bigram_count[(tok[i],tok[i+1])] = 1

bigrams = set(list(bigram_count.keys()))

100%|████████████████████████████████████| 4287/4287 [00:00<00:00, 48998.64it/s]


In [ ]:
#unsmoothed bigram probabilites

unsmoothed = dict()

for i in bigrams:
    countb = bigram_count[i]#total number of bigram count 
    countu = unigram_count[i[0]]#total number of unigram count of previous word
    unsmoothed[i] = countb/countu



In [ ]:
#smoothed bigram probability of the bigrams whose count is not equal to 0
smoothed = dict()
for i in bigrams:
    countb_s = bigram_count[i] + 1
    countu_s = unigram_count[i[0]]+V
    smoothed[i] = countb_s/countu_s
    
helper = dict()
for i in vocab:
    helper[i] = 1/(unigram_count[i]+V)

In [ ]:
#building the laplace model
laplace = dict()
for v in tqdm.tqdm(list(vocab)):
    for v1 in list(vocab):
        big = (v,v1)
        if(big in bigrams):
            laplace[big]= smoothed[big]#if the bigram exists in the corpus
        else:
            laplace[big] = helper[big[0]]#if the bigram doesn't exist then laplace depends only on previous word

100%|██████████████████████████████████████| 7950/7950 [00:59<00:00, 133.84it/s]


## Save smoothed bigram model

In [ ]:
# saving the smoothed bigram model


with open('laplace_final.pickle', 'wb') as handle:
    pickle.dump(laplace, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# laplace = unserialized_data

In [ ]:
len(laplace)

63202500

In [ ]:
#Segregating the positive and negative data
datapos = data[data['LABEL']==1]
dataneg = data[data['LABEL']==0]

In [ ]:
#finding the unigram count and bigram count of the words in positive data i.e. datapos
pos_unigram = dict()
pos_bigram = dict()
for i in tqdm.tqdm(datapos['TEXT']):
    token = i.split(' ')
    for i in token:
        if(i=="''"):#useless word removed
            continue
        try:
            pos_unigram[i]+=1
        except:
            pos_unigram[i] = 1
            
            
for i in tqdm.tqdm(datapos['TEXT']):
    token = i.split(' ')
    for i in range(len(token)-1):
        if(i == "''"): #useless word removed
            continue
        try:
            pos_bigram[(token[i],token[i+1])]+=1
        except:
            pos_bigram[(token[i],token[i+1])] = 1
            

100%|███████████████████████████████████| 2287/2287 [00:00<00:00, 113748.05it/s]


In [ ]:
#finding the unigram and bigram count of the words in negative data i.e. dataneg
neg_unigram = dict()
neg_bigram = dict()
for i in tqdm.tqdm(dataneg['TEXT']):
    token = i.split(' ')
    for i in token:
        if(i=="''"):
            continue
        try:
            neg_unigram[i]+=1
        except:
            neg_unigram[i] = 1
            
            
for i in tqdm.tqdm(dataneg['TEXT']):
    token = i.split(' ')
    for i in range(len(token)-1):
        if(i=="''"):
            continue
        try:
            neg_bigram[(token[i],token[i+1])]+=1
        except:
            neg_bigram[(token[i],token[i+1])] = 1
            

100%|███████████████████████████████████| 2000/2000 [00:00<00:00, 106010.46it/s]


In [ ]:
# saving the positive and negative bigram model



with open('positive_bigram.pickle', 'wb') as handle:
    pickle.dump(pos_bigram, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('negative_bigram.pickle', 'wb') as handle:
    pickle.dump(neg_bigram, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
#storing the starting words in negative  and positive sentiment sentences in training corpus
start_pos = dict()

for i in datapos['TEXT']:
    try:
        start_pos[i.split()[1]] = pos_unigram[i.split()[1]]/unigram_count[i.split()[1]]
    except:
        pass
        

start_neg = dict()

for i in dataneg['TEXT']:
    start_neg[i.split()[1]] = neg_unigram[i.split()[1]]/unigram_count[i.split()[1]]
    


In [ ]:
# S_pos = list(dict(sorted(start_pos.items(), key = lambda item : item[1], reverse=True)).keys())
# S_neg = list(dict(sorted(start_neg.items(), key = lambda item : item[1], reverse=True)).keys())


In [ ]:
#
def candidate_words(prev):
    candidates = dict()
    
    for i in list(bigrams):
        # print(i[0])
        if(prev==i[0]):
            candidates[i] = laplace[i]
    
    candidates = dict(sorted(candidates.items(),key = lambda item:item[1],reverse = True))
    
    return candidates

In [ ]:
def best_word(cw,ind,sentiment,prev):
    if(sentiment=='pos'):
        ugram = pos_unigram
        bgram = pos_bigram
    else:
        ugram = neg_unigram
        bgram = neg_bigram
        
    count = 0
    final_prob = dict()
        
    for i in cw.keys():
        if(count >=10):
            break
        else:
            count=count +1
        n1 = bigram_count[(prev,i[1])]
        try:
            n2 = bgram[(prev,i[1])]
        except:
            n2=0
            
        d1 = unigram_count[prev]
        try:
            d2 = ugram[prev]
        except:
            d2 = d1
        final_prob[i] = (n1+n2+1)/(d1+d2+V)
    
    B = dict(sorted(final_prob.items(),key = lambda item:item[1],reverse = True))

    return B
    
    
    

In [ ]:
def generating_sentences():
    final = []
    # sent = ['<s>']
    
    j = 0
    start_words = list()
    while len(final)<500:

        if(len(final)!=j):
            j-=1
        sent = ['<s>'] #start of the sentence
        num_words = 1
        sentiment = random.choice(['pos','neg'])#randomly choose which sentiment sentence should be generated
        for i in range(0,24):

            if(sent[-1]=='<s>'): 
                if(sentiment=='pos'):
                    x = random.choice(S_pos)#randomly choose a starting word from the list of positive sentiment start word list
                    sent.append(x)
                else:
                    y = random.choice(S_neg)#randomly choose a starting word from the list of negative sentiment start word list
                    sent.append(y)
                continue
            cw = candidate_words(sent[-1]) #sorted dictionary of all bigrams with previous word = the last word added to the sentence
            next_word = best_word(cw,i,sentiment,sent[-1])#getting the dictionary of words with beta probability
            k = list(next_word.keys())[:2] #slicing the dictionary to get the best 2 words
            v = list(next_word.values())[:2]

            try:
            
                index = random.randint(0,len(k)) #choosing a random word out of the best 2 words
                nw = k[index][1]

            except:
                continue 
            if(nw == '</s>' and num_words<12): # if the word found is </s> but the number of words <12(lower bound) then add more words
                i-=1
                continue
            elif(nw=='</s>'):

                p = " ".join(sent[1:]) # end sentence and add it to final list
                final.append(p)
                break
            else:
                num_words+=1 #store the number of words added so far
                sent.append(nw) #append word to sentences

        
    return final

In [ ]:
# sentences = generating_sentences() 
#retrieve already generated sentences to save time
sent = pd.read_csv("sentences.csv")
sentences = sent['sentences']
pred = sent['labels']

In [ ]:
len(sentences)

500

### 5.B Save the generated 500 sentences and their sentiment labels in csv

In [ ]:
import numpy as np
import csv
prim = []
# sec = []
for i in sentences:
    sec = []
    sec.append(i)
    prim.append(sec)

file = open('sentences.csv','w+',newline = '')
header = ["sentences"]

with file:
    write = csv.writer(file)
    write.writerow(header)
    write.writerows(prim)

X = pd.read_csv("sentences.csv")
X['labels'] = pred
X.to_csv("sentences.csv")

In [ ]:
pred = []
sid = SentimentIntensityAnalyzer()
for i in sentences:
    polarity = (sid.polarity_scores(i)['compound'])
    if(polarity<0):
        pred.append(0)
    else:
        pred.append(1)

## Report the Top -4 bigrams and their scores after smoothing

### 5.c Report the average Perplexity of the generated 500 sentence

## B.d Report 10 generated samples 5 positive + 5 negatives

In [ ]:
count = 0
for i in list(zip(pred,sentences)):
    if(count == 5):
        break
    if(i[0] ==1):
        print(i[1])
        count+=1

weeeell sure 'm going bed soon 'll see 'm sorry hear 're still
luving sunshine little bit disappointed weather today 'm going back work tomorrow hopefully
lmfaooo would n't think 'm sorry hear 're going sleep night sweet
friday night friends 'm going back work tomorrow hopefully 'll take care much
hayfever medication n't get ready work tomorrow hopefully 'll take care much


In [ ]:
#negative sentiment sentences
count = 0
for i in list(zip(pred,sentences)):
    if(count == 5):
        break
    if(i[0] ==0):
        print(i[1])
        count+=1

busy day 'm sorry hear 're going bed soon 'll try go
hates hates hates hates hates hates hates hates hates hates hates hates hates
despite last day 'm sorry hear 're going bed soon 'll try go
sushi 've missed tweets set today 'm going bed soon 'll see
whedonverse bloody wales however may pdt ñ load updates seems working hard


In [ ]:
#Reporting the accuracy of test set 
from sklearn.linear_model import LogisticRegression
data2 = pd.read_csv("A1_dataset.csv")
test_data = pd.read_csv("A2_test_dataset.csv")
def train_and_evaluate(train_sentences, train_labels,
test_sentences, test_labels):

    model = make_pipeline(TfidfVectorizer(), LogisticRegression())
    model.fit(train_sentences, train_labels)
    predicted_test_labels = model.predict(test_sentences)
    return accuracy_score(test_labels, predicted_test_labels)

train_sentences = data2['TEXT'].values
train_labels = data2['LABEL'].values
test_sentences = test_data['TEXT'].values
test_labels = test_data['LABEL'].values

## A.c Report the accuracy of test set using dataset A for training

In [ ]:
train_and_evaluate(train_sentences,train_labels,test_sentences,test_labels)

0.8773291925465838

In [ ]:
##CREATING DATASET B

In [ ]:
trial_data = data2[['LABEL','TEXT']] 
A = dict(zip(sentences,pred))
index = trial_data.shape[0]

In [ ]:
for key,val in A.items():
    lst = []
    lst.append(val)
    lst.append(key)
    trial_data.loc[index] = lst
    index+=1

In [ ]:
trial_data.shape

(4783, 2)

In [ ]:
## B.e REPORT THE ACCURACY OF THE TEST SET USING DATASET B FOR TRAINING

In [ ]:
train_and_evaluate(trial_data['TEXT'].values,trial_data['LABEL'].values,test_sentences,test_labels)

0.8819875776397516